In [34]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import skfolio
from skfolio import Population, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.optimization import InverseVolatility, MeanRisk, ObjectiveFunction
from skfolio.preprocessing import prices_to_returns
from data_loader import index_history
from data_loader import ticker_prices
from plotly.io import show
import requests

import warnings
warnings.filterwarnings('ignore')

In [35]:
# Определяем параметры
#tickers = ['AFKS', 'AFLT', 'ALRS', 'ASTR', 'BSPB', 'CBOM', 'CHMF', 'ENPG', 'FEES', 'FLOT', 'GAZP', 'GMKN', 'HEAD', 'HYDR', 'IRAO', 'LKOH', 'MAGN', 'MDMG', 'MOEX', 'MSNG', 'MTLR', 'MTSS', 'NLMK', 'NVTK', 'PHOR', 'PIKK', 'PLZL', 'POSI', 'RENI', 'ROSN', 'RTKM', 'RUAL', 'SBER', 'SBERP', 'SELG', 'SNGS', 'SNGSP', 'SVCB', 'T', 'TATN', 'TATNP', 'TRNFP', 'UGLD', 'UPRO', 'VKCO', 'VTBR', 'YDEX']
tickers = ['MCFTR', 'RGBITR']
risk_free_rate = 0.02 / 252
end_date = datetime.today()
start_date = end_date - timedelta(days=15 * 365)

In [36]:
# Загружаем исторические данные
data = index_history(tickers, start_date=start_date, end_date=end_date)
#data = ticker_prices(tickers)
data = data.set_index('TRADEDATE')

In [37]:
#data_filtered = data.drop(['YDEX', 'HEAD', 'SVCB', 'UGLD', 'ASTR'], axis=1)
data_filtered = data.copy()

In [38]:
# Convert prices to returns
returns = prices_to_returns(data_filtered, log_returns=True, fill_nan=False)

model = MeanRisk(
    risk_measure=RiskMeasure.STANDARD_DEVIATION,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    portfolio_params=dict(name="Max Sharpe"),
    risk_free_rate=risk_free_rate,
    #min_weights=0.0,

)

model.fit(returns)

MeanRisk(objective_function=MAXIMIZE_RATIO,
         portfolio_params={'name': 'Max Sharpe'},
         risk_free_rate=7.936507936507937e-05, risk_measure=Standard Deviation)

In [39]:
# Run optimization
model.fit(returns)

MeanRisk(objective_function=MAXIMIZE_RATIO,
         portfolio_params={'name': 'Max Sharpe'},
         risk_free_rate=7.936507936507937e-05, risk_measure=Standard Deviation)

In [40]:
print("Optimal Weights:")
for ticker, weight in zip(tickers, model.weights_):
    print(f"{ticker}: {weight:.4f}")

Optimal Weights:
MCFTR: 0.0933
RGBITR: 0.9067


In [41]:
benchmark = InverseVolatility(portfolio_params=dict(name="Inverse Vol"))
benchmark.fit(returns)

InverseVolatility(portfolio_params={'name': 'Inverse Vol'})

In [42]:
model_pred = model.predict(returns)
benchmark_pred = benchmark.predict(returns)

In [43]:
population = Population([model_pred, benchmark_pred])

In [44]:
fig = population.plot_cumulative_returns()
# show(fig) is only used for the documentation sticker.
show(fig)

In [51]:
population.summary()

,Max Sharpe,Inverse Vol
Mean,0.026%,0.029%
Annualized Mean,6.62%,7.32%
Variance,0.0030%,0.0043%
Annualized Variance,0.74%,1.08%
Semi-Variance,0.0019%,0.0029%
Annualized Semi-Variance,0.49%,0.72%
Standard Deviation,0.54%,0.66%
Annualized Standard Deviation,8.62%,10.40%
Semi-Deviation,0.44%,0.54%
Annualized Semi-Deviation,7.00%,8.51%


In [46]:
population.composition()

,Max Sharpe,Inverse Vol
asset,,
RGBITR,0.906653,0.745425
MCFTR,0.093347,0.254575


In [47]:
returns.corr()

,MCFTR,RGBITR
MCFTR,1.000000,0.494784
RGBITR,0.494784,1.000000


In [52]:
# Calculate the portfolio returns
portfolio_returns = returns.dot(model.weights_)

# Calculate cumulative returns
cumulative_returns = (1 + portfolio_returns).cumprod() - 1

# Calculate average return, cumulative return, and standard deviation
average_return = portfolio_returns.mean() * 252  # Annualized average return
cumulative_return = cumulative_returns.iloc[-1]  # Total cumulative return
portfolio_volatility = portfolio_returns.std() * np.sqrt(252)  # Annualized volatility

# Calculate the Sharpe Ratio
sharpe_ratio_portfolio = (average_return - risk_free_rate) / portfolio_volatility

# Print the results
print(f"Average Annual Return: {average_return:.4f}")
print(f"Cumulative Return: {cumulative_return:.4f}")
print(f"Portfolio Volatility: {portfolio_volatility:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio_portfolio:.4f}")


Average Annual Return: 0.0662
Cumulative Return: 1.2521
Portfolio Volatility: 0.0862
Sharpe Ratio: 0.7669


In [53]:
((portfolio_returns.mean() - (risk_free_rate/252)) / portfolio_returns.std())

np.float64(0.04831136478167754)

In [54]:
# Используем веса из библиотеки, но свою безрисковую ставку
skfolio_weights = model.weights_
portfolio_returns_skfolio = returns.dot(skfolio_weights)
sharpe_ratio_adjusted = ((portfolio_returns_skfolio.mean() * 252) - risk_free_rate) / (portfolio_returns_skfolio.std() * np.sqrt(252))

In [55]:
sharpe_ratio_adjusted

np.float64(0.7669191402626581)

In [56]:
# Используем веса из библиотеки, но свою безрисковую ставку
skfolio_weights = model.weights_
portfolio_returns_skfolio = returns.dot(skfolio_weights)
sharpe_ratio_adjusted = ((portfolio_returns_skfolio.mean()) - risk_free_rate/252) / (portfolio_returns_skfolio.std())
sharpe_ratio_adjusted

np.float64(0.04831136478167754)

In [59]:
portfolio_returns_skfolio.mean() * 252

np.float64(0.06622088953487801)